In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

# 影像数目的分布，2000-2020，堆积柱状图

In [3]:
# 大庆市
roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
# 天津市
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
# 武汉市
# roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
# 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
# 青藏高原
# roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [ ]:
a = ee.List.sequence(2000,2020,1)
# landsat5
def l5(year):
    start = ee.Algorithms.String(ee.Number(year).int()).cat(ee.String('-01-01'))
    end = ee.Algorithms.String(ee.Number(year).int()).cat(ee.String('-12-31'))
    dataset = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterBounds(roi).filterDate(start, end)
    return dataset.size()
print(a.map(l5).getInfo())

# landsat7
def l7(year):
    start = ee.Algorithms.String(ee.Number(year).int()).cat(ee.String('-01-01'))
    end = ee.Algorithms.String(ee.Number(year).int()).cat(ee.String('-12-31'))
    dataset = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').filterBounds(roi).filterDate(start, end)
    return dataset.size()
print(a.map(l7).getInfo())

# landsat8
def l8(year):
    start = ee.Algorithms.String(ee.Number(year).int()).cat(ee.String('-01-01'))
    end = ee.Algorithms.String(ee.Number(year).int()).cat(ee.String('-12-31'))
    dataset = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(roi).filterDate(start, end)
    return dataset.size()
print(a.map(l8).getInfo())

In [ ]:
a = ee.List.sequence(2000,2020,1)
l5 = a.map(l5).getInfo()
l7 = a.map(l7).getInfo()
l8 = a.map(l8).getInfo()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
x = np.arange(2000,2021)
plt.figure(figsize=(8, 6))
plt.bar(x, l5, width=0.45,align="center",label="Landsat5")
plt.bar(x, l7, width=0.45, align="center", bottom=l5, label="Landsat7")
plt.bar(x, l8, width=0.45, align="center", bottom=l7, label="Landsat8")

plt.ylim(0,280)
plt.xlabel("Year",size=18)
plt.ylabel("Number of images",size=18)
plt.tick_params(labelsize=15)
plt.xticks([2000,2005,2010,2015,2020])
plt.yticks([50,100,150,200,250])
plt.legend(prop={'size':13})
plt.title('Region A',size=20)

plt.show()

# 有效像元分布，2000-2020

## 计算

In [ ]:
# 大庆市
# roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
# 天津市
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
# 武汉市
roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
# 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
# 青藏高原
# roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [ ]:
# 移除无效像元
elevation = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
def maskSR(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    azimuth = image.get('SOLAR_AZIMUTH_ANGLE')
    zenith = image.get('SOLAR_ZENITH_ANGLE')
    return image.updateMask(mask).updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True))
# 图像可视化参数
visParams = {'bands': ['B5', 'B4', 'B3'],'min': 0,'max': 3000,'gamma': 1.4}

In [ ]:
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
       .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi) \
       .filterDate('2013-01-01', '2020-12-31') \
       .map(maskSR)
l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
       .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi) \
       .filterDate('2000-01-01', '2020-12-31')\
       .map(maskSR)          
l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
       .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi) \
       .filterDate('2000-01-01', '2012-12-31') \
       .map(maskSR)
print(l8.size().getInfo())
print(l7.size().getInfo())
print(l5.size().getInfo())
landsat_images =l8.merge(l7).merge(l5)
print(landsat_images.size().getInfo())
Map.addLayer(l8.map(maskSR).median().clip(roi), visParams,'image mosaic')

In [ ]:
validPixel = ee.ImageCollection(landsat_images).count().select('B3').clip(roi).rename('count')
Map.addLayer(validPixel, {'palette':['white','blue'],'min':0,'max':1000},'validPixel')

In [ ]:
task = ee.batch.Export.image.toAsset(**{
    'image': validPixel,
    'description': 'validPixel',
    'assetId': 'users/311605001111/validPixel_wuhan',
    'scale': 30,
    'region': roi,
    'maxPixels':1e13,
})
task.start()

## 制图

In [ ]:
## 大庆
validPixel = ee.Image('users/311605001111/validPixel_daqing')
roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
minmax = validPixel.reduceRegion(**{
    'reducer': ee.Reducer.minMax(),
    'geometry': roi,
    'scale': 30,
    'maxPixels': 1e14,
})
print(minmax.getInfo())

# 有效像元
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [123.9355, 45.8362,125.3055, 46.8762] 
fig = plt.figure(figsize=(10, 6))
visParams = {'min': 0.0,'max': 1300}

ax = cartoee.get_map(validPixel,cmap="rainbow",region=region,vis_params=visParams)
# cartoee.add_colorbar(ax,vis_params=visParams,cmap="rainbow",loc="right",label="number",orientation="vertical")
cartoee.add_gridlines(ax, interval=[0.3,0.3], linestyle=":")
ax.set_title(label = 'Region A', fontsize=18)

In [ ]:
## 天津
validPixel = ee.Image('users/311605001111/validPixel_tianjin')
roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
minmax = validPixel.reduceRegion(**{
    'reducer': ee.Reducer.minMax(),
    'geometry': roi,
    'scale': 30,
    'maxPixels': 1e14,
})
print(minmax.getInfo())

# 有效像元
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [115.9244, 38.6272,117.4944,39.6372] 
fig = plt.figure(figsize=(10, 8))
visParams = {'min': 0.0,'max': 1300}

ax = cartoee.get_map(validPixel,cmap="rainbow",region=region,vis_params=visParams)
# cartoee.add_colorbar(ax,vis_params=visParams,cmap="rainbow",loc="right",label="number",orientation="vertical")
cartoee.add_gridlines(ax, interval=[0.3,0.3], linestyle=":")
ax.set_title(label = 'Region B', fontsize=18)

In [ ]:
## 武汉
validPixel = ee.Image('users/311605001111/validPixel_wuhan')
roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
minmax = validPixel.reduceRegion(**{
    'reducer': ee.Reducer.minMax(),
    'geometry': roi,
    'scale': 30,
    'maxPixels': 1e14,
})
print(minmax.getInfo())

# 有效像元
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [113.7393, 29.8642,115.0993, 30.9242] 
fig = plt.figure(figsize=(10, 6))
visParams = {'min': 0.0,'max': 1300}

ax = cartoee.get_map(validPixel,cmap="rainbow",region=region,vis_params=visParams)
# cartoee.add_colorbar(ax,vis_params=visParams,cmap="rainbow",loc="right",label="number",orientation="vertical")
cartoee.add_gridlines(ax, interval=[0.3,0.3], linestyle=":")
ax.set_title(label = 'Region C', fontsize=18)

In [ ]:
## 珠三角
validPixel = ee.Image('users/311605001111/validPixel_PRD')
roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
minmax = validPixel.reduceRegion(**{
    'reducer': ee.Reducer.minMax(),
    'geometry': roi,
    'scale': 30,
    'maxPixels': 1e14,
})
print(minmax.getInfo())

# 有效像元
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [112.7614, 22.2347,114.0514, 23.2547] 
fig = plt.figure(figsize=(10, 6))
visParams = {'min': 0.0,'max': 1300}

ax = cartoee.get_map(validPixel,cmap="rainbow",region=region,vis_params=visParams)
# cartoee.add_colorbar(ax,vis_params=visParams,cmap="rainbow",loc="right",label="number",orientation="vertical")
cartoee.add_gridlines(ax, interval=[0.3,0.3], linestyle=":")
ax.set_title(label = 'Region D', fontsize=18)

In [ ]:
## 青藏高原
validPixel = ee.Image('users/311605001111/validPixel_Tibetan')
roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
minmax = validPixel.reduceRegion(**{
    'reducer': ee.Reducer.minMax(),
    'geometry': roi,
    'scale': 30,
    'maxPixels': 1e14,
})
print(minmax.getInfo())

# 有效像元
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [89.8826, 34.6579,91.3626, 35.8279] 
fig = plt.figure(figsize=(10, 6))
visParams = {'min': 0.0,'max': 1300}

ax = cartoee.get_map(validPixel,cmap="rainbow",region=region,vis_params=visParams)
# cartoee.add_colorbar(ax,vis_params=visParams,cmap="rainbow",loc="right",label="number",orientation="vertical")
cartoee.add_gridlines(ax, interval=[0.3,0.3], linestyle=":")
ax.set_title(label = 'Region E', fontsize=18)